# Livestock Database - Text-to-SQL AI Agent

In [1]:
# Cell 1: Imports
import os
import pandas as pd
from typing import Optional, Dict, Any, List
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
import pymssql

load_dotenv()
print("✓ Imports successful")

✓ Imports successful


In [2]:
# Cell 2: Configuration
ALLOWED_TABLES = [
    "Peoplebreedlookup",
    "Speciesavailable",
    "Speciesbreedlookuptable",
    "Speciescategory",
    "Speciescolorlookuptable",
    "Speciespatternlookuptable",
    "Speciesregistrationtypelookuptable"
]

DB_HOST = os.getenv("DB_HOST", "").strip()
DB_PORT = int(os.getenv("DB_PORT", "1433").strip())
DB_USER = os.getenv("DB_USER", "").strip()
DB_PASSWORD = os.getenv("DB_PASSWORD", "")
DB_NAME = os.getenv("DB_NAME", "").strip()

print(f"Host: {DB_HOST}")
print(f"Port: {DB_PORT}")
print(f"User: {DB_USER}")
print(f"Database: {DB_NAME}")

Host: 34.70.16.88
Port: 1433
User: sqlserver
Database: oatmealailivedb


In [3]:
# Cell 3: Connect to SQL Server
print("Connecting to SQL Server...")

connection = pymssql.connect(
    server=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME,
    as_dict=True
)

print(f"✓ Connected to SQL Server: {DB_NAME}")

Connecting to SQL Server...
✓ Connected to SQL Server: oatmealailivedb


In [4]:
# Cell 4: Test - List tables
cursor = connection.cursor()

print("📊 Checking tables...\n")
for table in ALLOWED_TABLES:
    try:
        cursor.execute(f"SELECT COUNT(*) as cnt FROM {table}")
        result = cursor.fetchone()
        print(f"✓ {table}: {result['cnt']} rows")
    except Exception as e:
        print(f"✗ {table}: {e}")

📊 Checking tables...

✓ Peoplebreedlookup: 0 rows
✓ Speciesavailable: 30 rows
✓ Speciesbreedlookuptable: 2512 rows
✓ Speciescategory: 159 rows
✓ Speciescolorlookuptable: 303 rows
✓ Speciespatternlookuptable: 47 rows
✓ Speciesregistrationtypelookuptable: 89 rows


In [6]:
# Cell 5: Database Manager Class
class DatabaseManager:
    def __init__(self, conn, allowed_tables):
        self.connection = conn
        self.allowed_tables = [t.lower() for t in allowed_tables]
    
    def execute(self, query):
        # Security check
        import re
        query_lower = query.lower()
        tables = re.findall(r'from\s+\[?(\w+)\]?', query_lower)
        tables += re.findall(r'join\s+\[?(\w+)\]?', query_lower)
        
        for t in tables:
            if t not in self.allowed_tables:
                raise PermissionError(f"Access denied to table: {t}")
        
        cursor = self.connection.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        return pd.DataFrame(results) if results else pd.DataFrame()
    
    def get_schema(self):
        schema = []
        cursor = self.connection.cursor()
        for table in ALLOWED_TABLES:
            cursor.execute(f"""
                SELECT COLUMN_NAME, DATA_TYPE 
                FROM INFORMATION_SCHEMA.COLUMNS 
                WHERE TABLE_NAME = '{table}'
            """)
            cols = cursor.fetchall()
            if cols:
                schema.append(f"-- {table}")
                for c in cols:
                    schema.append(f"   {c['COLUMN_NAME']} ({c['DATA_TYPE']})")
        return "\n".join(schema)

db = DatabaseManager(connection, ALLOWED_TABLES)
print("✓ Database Manager ready")
print("\nSchema:")
print(db.get_schema())

✓ Database Manager ready

Schema:
-- Peoplebreedlookup
   PeopleBreedLookupID (int)
   PeopleID (int)
   BreedID (int)
   SpeciesID (int)
-- Speciesavailable
   SpeciesID (int)
   Species (varchar)
   PreferedspeciesID (int)
   SpeciesPriority (int)
   SpeciesAvailable (smallint)
   SpeciesSalesType (varchar)
   MaleTerm (varchar)
   FemaleTerm (varchar)
   SireTerm (varchar)
   Damterm (varchar)
   StudTerm (varchar)
   SingularTerm (varchar)
   PluralTerm (varchar)
   BabyTerm (varchar)
   GeldingTermMale (varchar)
   GeldedtermFemale (varchar)
   TagID (varchar)
   Fibertype (varchar)
   Brand (varchar)
   Waddle (varchar)
   Ancestryterm (varchar)
   Thyroid (varchar)
   Hip (varchar)
   Pulled (smallint)
   GestationPeriod (int)
   SpeciesAvailableonSite (smallint)
   SpeciesText1 (varchar)
   SpeciesImage1 (varchar)
   Speciesvideo1 (varchar)
   SpeciesText2 (varchar)
   SpeciesImage2 (varchar)
   Speciesvideo2 (varchar)
   SpeciesText3 (varchar)
   SpeciesImage3 (varchar)
   Spe

In [7]:
# Cell 6: AI Agent Setup
class SQLQuery(BaseModel):
    sql: str = Field(description="SQL query")
    explanation: str = Field(description="What the query does")
    is_safe: bool = Field(description="True if SELECT only")

# Use Google API Key (not Vertex AI)
api_key = os.getenv("GOOGLE_API_KEY", "").strip()
print(f"Using API Key: {api_key[:10]}...")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    google_api_key=api_key
)
structured_llm = llm.with_structured_output(SQLQuery)

print("✓ AI Agent ready")

Using API Key: AIzaSyAUOc...
✓ AI Agent ready


In [8]:
# Cell 7: Ask Function
def ask(question: str):
    schema = db.get_schema()
    tables_list = ", ".join(ALLOWED_TABLES)
    
    prompt = f"""You are a SQL Server expert. Generate T-SQL for this question.

ALLOWED TABLES ONLY: {tables_list}

Schema:
{schema}

Question: {question}

Rules:
- Use TOP instead of LIMIT
- Only SELECT queries
- Only use allowed tables
"""
    
    result = structured_llm.invoke(prompt)
    
    print(f"\n📝 SQL: {result.sql}")
    print(f"💡 {result.explanation}")
    
    if result.is_safe:
        try:
            df = db.execute(result.sql)
            print(f"\n✅ {len(df)} rows returned")
            return df
        except Exception as e:
            print(f"\n❌ Error: {e}")
            return None
    else:
        print("\n⚠️ Query not safe")
        return None

print("✓ ask() function ready")

✓ ask() function ready


In [9]:
# Cell 8: Try it!
df = ask("Show me all species categories")
if df is not None:
    display(df)


📝 SQL: SELECT TOP (1000) SpeciesCategory FROM SpeciesCategory
💡 This SQL query selects the 'SpeciesCategory' column from the 'SpeciesCategory' table and retrieves the top 1000 rows. This effectively shows all species categories.

✅ 159 rows returned


,SpeciesCategory
0,Maiden
1,Dam
2,Stud
3,Gelding
4,Heifer
...,...
154,Fawn
155,Bull
156,Cow
157,Calf


In [9]:
# Cell 9: Your questions
df = ask("What colors are available?")
if df is not None:
    display(df)


📝 SQL: SELECT DISTINCT SpeciesColor FROM Speciescolorlookuptable
💡 This SQL query selects all distinct species colors from the Speciescolorlookuptable.

✅ 211 rows returned


,SpeciesColor
0,Agouti
1,Albino
2,Apricot
3,Barred
4,Bay
...,...
206,Wheaten
207,White
208,White Laced Red
209,White/Tan


In [10]:

df = ask("What breeds are in the breed lookup table?")
df = ask("Count records in each table")
df = ask("Show me all colors and patterns")


📝 SQL: SELECT DISTINCT Breed FROM Speciesbreedlookuptable
💡 This SQL query selects all distinct breed names from the Speciesbreedlookuptable.

✅ 2402 rows returned

📝 SQL: SELECT 'Peoplebreedlookup' AS TableName, COUNT(*) AS RecordCount FROM Peoplebreedlookup
UNION ALL
SELECT 'Speciesavailable' AS TableName, COUNT(*) AS RecordCount FROM Speciesavailable
UNION ALL
SELECT 'Speciesbreedlookuptable' AS TableName, COUNT(*) AS RecordCount FROM Speciesbreedlookuptable
UNION ALL
SELECT 'Speciescategory' AS TableName, COUNT(*) AS RecordCount FROM Speciescategory
UNION ALL
SELECT 'Speciescolorlookuptable' AS TableName, COUNT(*) AS RecordCount FROM Speciescolorlookuptable
UNION ALL
SELECT 'Speciespatternlookuptable' AS TableName, COUNT(*) AS RecordCount FROM Speciespatternlookuptable
UNION ALL
SELECT 'Speciesregistrationtypelookuptable' AS TableName, COUNT(*) AS RecordCount FROM Speciesregistrationtypelookuptable;
💡 This SQL query counts the number of records in each of the allowed tables and re

In [12]:
display(df)

,SpeciesColor
0,Appaloosa
1,Apricot
2,Bay
3,Bay Black
4,Beige
...,...
107,Tuxedo
108,Wheaten
109,White
110,Whitelies
